# Retail Dataset

This dataset contains lot of historical sales data. It was extracted from a Brazilian top retailer and has many SKUs and many stores. The data was transformed to protect the identity of the retailer.

## 1. Load 

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import calendar
import datetime  
from datetime import datetime as dt
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [ ]:
retail = pd.read_csv('mock_kaggle.csv')

In [ ]:
retail['date'] = pd.to_datetime(retail['date'])

<blank>

<blank>

<blank>

## 2. Explorer Dataset

In [ ]:
retail.head()

In [ ]:
retail.tail()

In [ ]:
retail.info()

In [ ]:
retail.describe()

In [ ]:
retail.isnull().sum()

In [ ]:
retail['dow'] = retail['date'].dt.dayofweek
retail['day_of_week'] = retail['date'].dt.day_name()
#retail['weeknum'] = retail['date'].dt.week
retail['weeknum'] = retail['date'].dt.strftime('%W')

In [ ]:
retail.head(20)

### - Day of Week Analysis

In [ ]:
retail_dow = retail.groupby(['day_of_week','dow']).mean()

In [ ]:
retail['sale']

In [ ]:
retail_dow

In [ ]:
retail_dow.sort_values("dow", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [ ]:
retail_dow

In [ ]:
df = retail_dow.drop(['stock'], axis=1)

In [ ]:
df

In [ ]:
df["sale"].plot(label="Average sale",  title = "Relationship between price(t_avg) and sales(t_avg) - day of week", legend=True, figsize=(13,7))
df["price"].plot(label="Average price", legend=True, secondary_y=True)
## df.plot(figsize=(12,8),secondary_y=True)
plt.show()

## According to above dual weekly graph, we could find there is an inverse relationship between sales and price

In [ ]:
retail

In [ ]:
retail['year'] = retail['date'].dt.year

In [ ]:
retail.head()

In [ ]:
retail_dow2 = retail.groupby(['year','weeknum']).mean()

In [ ]:
retail_dow2

In [ ]:
df2 = retail_dow2.drop(['stock','dow'], axis=1)

In [ ]:
df2

In [ ]:
df2["sale"].plot(label="Average sale",  title = "Relationship between price(avg) and sales(avg) - weekly ", legend=True, figsize=(13,7))
df2["price"].plot(label="Average price", legend=True, secondary_y=True)
## df.plot(figsize=(12,8),secondary_y=True)
plt.show()

<blank>

<blank>

<blank>

# 3. Moving Average

In [ ]:
retail['7-day'] = retail['price'].rolling(7).mean()
retail['14-day'] = retail['price'].rolling(14).mean()
retail['21-day'] = retail['price'].rolling(21).mean()

In [ ]:
retail

In [ ]:
with plt.style.context('ggplot'):
    plt.figure(figsize = (20,10))
    plt.plot(retail.price[-120:], label = 'Real Price')
    plt.plot(retail['7-day'][-120:], label = '7 Day Moving Average')
    plt.plot(retail['14-day'][-120:], label = '14 Day Moving Average')
    plt.plot(retail['21-day'][-120:], label = '21 Day Moving Average')
    plt.legend(loc =1)

In [ ]:
retail['7-day'] = retail['sale'].rolling(7).mean()
retail['14-day'] = retail['sale'].rolling(14).mean()
retail['21-day'] = retail['sale'].rolling(21).mean()

In [ ]:
with plt.style.context('ggplot'):
    plt.figure(figsize = (20,10))
    plt.plot(retail.sale[-120:], label = 'Real Price')
    plt.plot(retail['7-day'][-120:], label = '7 Day Moving Average')
    plt.plot(retail['14-day'][-120:], label = '14 Day Moving Average')
    plt.plot(retail['21-day'][-120:], label = '21 Day Moving Average')
    plt.legend(loc =1)

In [ ]:
retail.head(20)

## Dicky Fuller Test

#### Check for stationarity using the Dicky Fuller Test for all three datasets


In [ ]:
from statsmodels.tsa.stattools import adfuller
result1 = adfuller(retail['price'])
print(result1)

<blank>

<blank>

<blank>

# 4.1 Simple Exponential Smoothing 

In [ ]:
def simple_exp_smooth(d,extra_periods,alpha):  


  d = np.array(d)  # Transform the input into a numpy array  

  cols = len(d)  # Historical period length  

  d = np.append(d,[np.nan]*extra_periods)  # Append np.nan into the demand array to cover future periods  



  f = np.full(cols+extra_periods,np.nan)  # Forecast array  

  f[0] = d[0]  # initialization of first forecast  

    

  # Create all the t+1 forecasts until end of historical period  

  for t in range(1,cols+1):  

    f[t] = alpha*d[t-1]+(1-alpha)*f[t-1]  
    f[cols+1:] = f[t]  # Forecast for all extra periods  


  df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Error":f-d}) 
  return df

# 4.2 Double Exponential Smoothing

In [ ]:
# Double Exponential Smoothing
def double_exp_smooth(d,extra_periods,alpha,beta):  

    d = np.array(d)  # Transform the input into a numpy array  
    cols = len(d)  # Historical period length  
    d = np.append(d,[np.nan]*extra_periods)  # Append np.nan into the demand array to cover future periods  
# Creation of the level, trend, and forecast arrays
    f= np.full(cols+extra_periods,np.nan)  # Forecast array  
    a = np.full(cols+extra_periods,np.nan)
    b = np.full(cols+extra_periods,np.nan)
# Level and trend initialization 
    a[0] = d[0]
    b[0] = d[1] -d[0]

    
  # Create all the t+1 forecasts until end of historical period  

    for t in range(1,cols):  
        f[t] = a[t-1]+ b[t-1]
        a[t] = alpha*d[t] + (1-alpha)*(a[t-1] + b[t-1])
        b[t] = beta*(a[t] - a[t-1]) + (1-beta)* b[t-1]

# Forecast for all extra periods  
    for t in range(cols, cols+extra_periods):
        f[t] = a[t-1] + b[t-1]
        a[t] = f[t]
        b[t] = b[t-1]


    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Level": a, "Trend": b, "Error":f-d}) 
    return df

In [ ]:
df1=simple_exp_smooth(retail['sale'],7,1)
df1.head(10)

In [ ]:
# For KPI Calculation
MAE = df1["Error"].abs().mean()  
print("MAE:",round(MAE,2)) 
RMSE = np.sqrt((df1["Error"]**2).mean())
print("RMSE:",round(RMSE,2))

In [ ]:
#For Plotting
df1.index.name = "Periods"
df1[["Demand","Forecast"]].plot(figsize=(20,8),title="Simple exponential smoothing",style=["-","--"])  
plt.show()


In [ ]:
df11=double_exp_smooth(retail['sale'],7,1,0.05)
df11.tail(10)

In [ ]:
MAE = df11["Error"].abs().mean()  
print("MAE:",round(MAE,2)) 
RMSE = np.sqrt((df11["Error"]**2).mean())
print("RMSE:",round(RMSE,2))

In [ ]:
#For Plotting
df11.index.name = "Periods"
df11[["Demand","Forecast"]].plot(figsize=(20,8),title="Double exponential smoothing",style=["-","--"])  
plt.show()

<blank>

<blank>
<blank>
<blank>

# 5. Machine Learning

In [ ]:
retail

Predict based on weekly average

In [ ]:
retail["year_week"] = retail["year"].map(str) + '-' + retail["weeknum"].map(str)

In [ ]:
retail

In [ ]:
retail_ml = retail.groupby(["year_week"]).mean()

In [ ]:
retail_ml

In [ ]:
retail_ml = retail_ml.drop(['stock','year','7-day','14-day','21-day','dow','price'], axis=1)

In [ ]:
retail_ml.sort_values("year_week", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 


In [ ]:
retail_ml

In [ ]:
retail_lstm = retail_ml

In [ ]:
df_ml = pd.DataFrame(retail_ml)

In [ ]:
df_ml

In [ ]:
df_ml = df_ml.sort_values(by = ['year_week'], ascending = [True])
retail_ml = retail_ml.sort_values(by = ['year_week'], ascending = [True])

In [ ]:
# Create 53 weeks of lag values to predict current observation
# Shift of 53 weeks in this case
for i in range(53,0,-1):
    df_ml[['t-'+str(i)]] = retail_ml.shift(i)
print(df_ml)

In [ ]:
df_ml

In [ ]:
retail.describe()

In [ ]:
retail.to_csv('output.csv')

In [ ]:
df_ml = df_ml.drop(['sale'], axis=1)

In [ ]:
df_ml

In [ ]:
# Create a new subsetted dataframe, removing Nans from first 12 rows
df_ml2 = df_ml[53:]
print(df_ml2)

In [ ]:
df_ml2.values

In [ ]:
# Split Data into dependent(target) and independent(features) variables

retail_ml = df_ml2.values
# Lagged variables (features) and original time series data (target)
X2 = retail_ml[:,0:]  # slice all rows and start with column 0 and go up to but not including the last column
y2 = retail_ml[:,0:]  # slice all rows and last column, essentially separating out 't' column

In [ ]:
# Columns t-1 to t-12, which are the lagged variables
X2.shape

In [ ]:
# Column t, which is the original time series
# Give first 10 values of target variable, time series
y2.shape

In [ ]:
X2

# We are using 80-20 and 70-30 split.

In [ ]:
# Target(Y) Train-Test split

Y2 = y2
traintarget_size = int(len(Y2) * 0.80)   # Set split
print(traintarget_size)
train_target, test_target = Y2[:traintarget_size], Y2[traintarget_size:len(Y2)]

print('Observations for Target: %d' % (len(Y2)))
print('Training Observations for Target: %d' % (len(train_target)))
print('Testing Observations for Target: %d' % (len(test_target)))

In [ ]:
Y2[traintarget_size:len(Y2)]

In [ ]:
# Features(X) Train-Test split

trainfeature_size = int(len(X2) * 0.80)
train_feature, test_feature = X2[:trainfeature_size], X2[trainfeature_size:len(X2)]
print('Observations for feature: %d' % (len(X2)))
print('Training Observations for feature: %d' % (len(train_feature)))
print('Testing Observations for feature: %d' % (len(test_feature)))

In [ ]:
train_feature

## Benchmark Model

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
  
reg = LinearRegression() # Create a linear regression object
  
reg = reg.fit(train_feature, train_target) # Fit it to the training data
  
# Create two predictions for the training and test sets
train_prediction = reg.predict(train_feature)
test_prediction = reg.predict(test_feature)

In [ ]:
train_prediction

In [ ]:
test_prediction

In [ ]:
# Compute the MAE for both the training and test sets

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,3))

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,3))



## Decision Tree Model

In [ ]:
# Decision Tree Regression Model

from sklearn.tree import DecisionTreeRegressor

# Create a decision tree regression model with default arguments
decision_tree_retail = DecisionTreeRegressor()  # max_depth not set

# Fit the model to the training features and targets
decision_tree_retail.fit(train_feature, train_target)

# Check the score on train and test
print(decision_tree_retail.score(train_feature, train_target))
print(decision_tree_retail.score(test_feature,test_target))  # predictions are horrible if negative value, no relationship if 0


In [ ]:
# Find Best Max Depth

# Loop through a few different max depths and check the performance
# Try different max depths. We want to optimize our ML models to make the best predictions possible.
# For regular decision trees, max_depth, which is a hyperparameter, limits the number of splits in a tree.
# You can find the best value of max_depth based on the R-squared score of the model on the test set.

for d in [2, 3,4, 5,7,8,10]:
    # Create the tree and fit it
    decision_tree_retail = DecisionTreeRegressor(max_depth=d)
    decision_tree_retail.fit(train_feature, train_target)

    # Print out the scores on train and test
    print('max_depth=', str(d))
    print(decision_tree_retail.score(train_feature, train_target))
    print(decision_tree_retail.score(test_feature, test_target), '\n')  # You want the test score to be positive
    
# R-square for train and test scores are below. 
;

### max_depth = 10 

In [ ]:
# Plot predicted against actual values

from matplotlib import pyplot as plt

# Use the best max_depth 
decision_tree_retail = DecisionTreeRegressor(max_depth=5) # Fill in best max depth score here
decision_tree_retail.fit(train_feature, train_target)

# Predict values for train and test
train_prediction = decision_tree_retail.predict(train_feature)

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,1))


test_prediction = decision_tree_retail.predict(test_feature)

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,1))

# Scatter the predictions vs actual values, orange is predicted
plt.scatter(train_prediction, train_target, label='train')  # blue 
plt.scatter(test_prediction, test_target, label='test')  # orange
plt.show()


<blank>

<blank>

## Random Forest Model

In [ ]:
# Random Forest Model
from sklearn.ensemble import RandomForestRegressor

# Create the random forest model and fit to the training data
rfr = RandomForestRegressor(n_estimators=200)
rfr.fit(train_feature, train_target)

# Look at the R^2 scores on train and test
print(rfr.score(train_feature, train_target))
print(rfr.score(test_feature, test_target))  # Try to attain a positive value

In [ ]:
from sklearn.model_selection import ParameterGrid
import numpy as np

# Create a dictionary of hyperparameters to search
# n_estimators is the number of trees in the forest. The larger the better, but also takes longer it will take to compute. 
# Run grid search
#grid = {'n_estimators': [200], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'random_state': [13]}
grid = {'n_estimators': [200], 'max_depth': [10], 'max_features': [3], 'random_state': [13]}
test_scores = []

# Loop through the parameter grid, set the hyperparameters, and save the scores
for g in ParameterGrid(grid):
    rfr.set_params(**g)  # ** is "unpacking" the dictionary
    rfr.fit(train_feature, train_target)
    test_scores.append(rfr.score(test_feature, test_target))

# Find best hyperparameters from the test score and print
best_idx = np.argmax(test_scores)
print(test_scores[best_idx], ParameterGrid(grid)[best_idx])  

# The best test score

### We use max_depth : 10

In [ ]:
# Use the best hyperparameters from before to fit a random forest model
rfr = RandomForestRegressor(n_estimators=200, max_depth=8, max_features = 3, random_state=13)
rfr.fit(train_feature, train_target)

# Make predictions with our model
train_prediction = rfr.predict(train_feature)
test_prediction = rfr.predict(test_feature)

# Create a scatter plot with train and test actual vs predictions
plt.scatter(train_target, train_prediction, label='train')
plt.scatter(test_target, test_prediction, label='test')
plt.legend()
plt.show()

In [ ]:
# Compute the MAE for both the training and test sets

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,1))

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,1))



In [ ]:
# Random Forest Feature Importance
# get column names
df_ml.columns

In [ ]:
df_ml

In [ ]:
# Get feature importances from our random forest model
importances = rfr.feature_importances_

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x = range(len(importances))

# Create tick labels 
feature_names = ['t-53','t-52','t-51','t-50','t-49','t-48','t-47','t-46','t-45','t-44','t-43','t-42','t-41','t-40','t-39','t-38','t-37','t-36','t-35','t-34','t-33','t-32','t-31','t-30','t-29','t-28','t-27','t-26','t-25','t-24','t-23','t-22','t-21','t-20','t-19','t-18','t-17','t-16','t-15','t-14','t-13','t-12','t-11','t-10','t-9','t-8','t-7','t-6','t-5','t-4','t-3','t-2','t-1']
labels = np.array(feature_names)[sorted_index]
plt.figure(figsize=(15, 3))
plt.bar(x, importances[sorted_index], tick_label=labels, width = 0.4)

# Rotate tick labels to vertical

plt.xticks(rotation=90)
plt.show()

In [ ]:
importances

<blank>

<blank>

<blank>

<blank>

<blank>

<blank>

<blank>

<blank>

<blank>

<blank>

## 70:30

In [ ]:
# Target(Y) Train-Test split

Y3 = y2
traintarget_size = int(len(Y3) * 0.7)   # Set split
print(traintarget_size)
train_target, test_target = Y3[:traintarget_size], Y3[traintarget_size:len(Y2)]

print('Observations for Target: %d' % (len(Y3)))
print('Training Observations for Target: %d' % (len(train_target)))
print('Testing Observations for Target: %d' % (len(test_target)))

In [ ]:
# Features(X) Train-Test split

trainfeature_size = int(len(X2) * 0.7)
train_feature, test_feature = X2[:trainfeature_size], X2[trainfeature_size:len(X2)]
print('Observations for feature: %d' % (len(X2)))
print('Training Observations for feature: %d' % (len(train_feature)))
print('Testing Observations for feature: %d' % (len(test_feature)))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
  
reg = LinearRegression() # Create a linear regression object
  
reg = reg.fit(train_feature, train_target) # Fit it to the training data
  
# Create two predictions for the training and test sets
train_prediction = reg.predict(train_feature)
test_prediction = reg.predict(test_feature)

In [ ]:
# Compute the MAE for both the training and test sets

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,3))

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,3))



## Decision Tree

In [ ]:
# Decision Tree Regression Model

from sklearn.tree import DecisionTreeRegressor

# Create a decision tree regression model with default arguments
decision_tree_avocado = DecisionTreeRegressor()  # max_depth not set

# Fit the model to the training features and targets
decision_tree_avocado.fit(train_feature, train_target)

# Check the score on train and test
print(decision_tree_avocado.score(train_feature, train_target))
print(decision_tree_avocado.score(test_feature,test_target))  # predictions are horrible if negative value, no relationship if 0


In [ ]:
# Find Best Max Depth

# Loop through a few different max depths and check the performance
# Try different max depths. We want to optimize our ML models to make the best predictions possible.
# For regular decision trees, max_depth, which is a hyperparameter, limits the number of splits in a tree.
# You can find the best value of max_depth based on the R-squared score of the model on the test set.

for d in [2, 3,4, 5,7,8,10]:
    # Create the tree and fit it
    decision_tree_avocado = DecisionTreeRegressor(max_depth=d)
    decision_tree_avocado.fit(train_feature, train_target)

    # Print out the scores on train and test
    print('max_depth=', str(d))
    print(decision_tree_avocado.score(train_feature, train_target))
    print(decision_tree_avocado.score(test_feature, test_target), '\n')  # You want the test score to be positive
    
# R-square for train and test scores are below. 

In [ ]:
# Plot predicted against actual values

from matplotlib import pyplot as plt

# Use the best max_depth 
decision_tree_avocado = DecisionTreeRegressor(max_depth=5) # Fill in best max depth score here
decision_tree_avocado.fit(train_feature, train_target)

# Predict values for train and test
train_prediction = decision_tree_avocado.predict(train_feature)

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,1))


test_prediction = decision_tree_avocado.predict(test_feature)

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,1))

# Scatter the predictions vs actual values, orange is predicted
plt.scatter(train_prediction, train_target, label='train')  # blue 
plt.scatter(test_prediction, test_target, label='test')  
plt.show()

## Random Forrest

In [ ]:
from sklearn.model_selection import ParameterGrid
import numpy as np

# Create a dictionary of hyperparameters to search
# n_estimators is the number of trees in the forest. The larger the better, but also takes longer it will take to compute. 
# Run grid search
#grid = {'n_estimators': [200], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'random_state': [13]}
grid = {'n_estimators': [200], 'max_depth': [8], 'max_features': [3], 'random_state': [13]}
test_scores = []

# Loop through the parameter grid, set the hyperparameters, and save the scores
for g in ParameterGrid(grid):
    rfr.set_params(**g)  # ** is "unpacking" the dictionary
    rfr.fit(train_feature, train_target)
    test_scores.append(rfr.score(test_feature, test_target))

# Find best hyperparameters from the test score and print
best_idx = np.argmax(test_scores)
print(test_scores[best_idx], ParameterGrid(grid)[best_idx])  

# The best test score

In [ ]:
# Compute the MAE for both the training and test sets

MAE_train=np.mean(abs(train_target-train_prediction))/np.mean(train_target)
print("Tree on train set MAE%:", round(MAE_train*100,1))

MAE_test=np.mean(abs(test_target-test_prediction))/np.mean(test_target)
print("Tree on test set MAE%:", round(MAE_test*100,1))



<blank>
    <blank>
        <blank>

# 6. LSTM

In [ ]:
import math
import matplotlib.pyplot as plt
from statsmodels.tools.eval_measures import rmse
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
retail_lstm

In [ ]:
retail_lstm['year_week'] = retail_lstm.index

In [ ]:
retail_lstm

In [ ]:
retail_lstm.reset_index(drop=True, inplace=True)

In [ ]:
retail_lstm.index = retail_lstm.year_week.str.replace('-', '')

In [ ]:
retail_lstm

In [ ]:
retail_lstm.rename(index={'year_week': 'year_weeks'})

In [ ]:
retail_lstm = retail_lstm.drop(['year_week'], axis=1) 
# (['year_week'], inplace=True)

In [ ]:
retail_lstm

In [ ]:
train, test = retail_lstm[:-12],retail_lstm[-12:]

In [ ]:
train

In [ ]:
test

In [ ]:
scalar =MinMaxScaler()
scalar.fit(train)
train = scalar.transform(train)
test = scalar.transform(test)

In [ ]:
test.shape

In [ ]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, n_input, batch_size =6)

# Epoch Start

# Epoch 50

In [ ]:
model = Sequential()
# Adding the input layer and LSTM layer
model.add(LSTM(200, activation= 'relu', input_shape =(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer ='adam', loss='mse')
history = model.fit_generator(generator, epochs = 50)

In [ ]:
pred_list =[]
batch = train[-n_input:].reshape(1, n_input, n_features)
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:], [[pred_list[i]]], axis=1)

pred_list

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.title('model loss - Epoch 50')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
df_predict= pd.DataFrame(scalar.inverse_transform(pred_list), index= retail_lstm[-n_input:].index, columns =['Predictions'])

In [ ]:
df_test = pd.concat([retail_lstm, df_predict], axis =1)

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel("Week")
plt.ylabel("Retail Sales")
plt.title("Retail Sales - Epoch 250")
plt.plot(df_test.index, df_test['sale'])
plt.plot(df_test.index, df_test['Predictions'], color ='r')


## RMSE (Epoch 50)

In [ ]:
testScore = math.sqrt(mean_squared_error(df_test.iloc[-12:,0].values, df_test.iloc[-12:,1].values))
print('Test Score: %.2f RMSE' % (testScore))

<blank>

<blank>

## Epoch 100

In [ ]:
model = Sequential()
# Adding the input layer and LSTM layer
model.add(LSTM(200, activation= 'relu', input_shape =(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer ='adam', loss='mse')
history = model.fit_generator(generator, epochs =100)

In [ ]:
pred_list =[]
batch = train[-n_input:].reshape(1, n_input, n_features)
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:], [[pred_list[i]]], axis=1)

pred_list

## Model Loss

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.title('model loss - Epoch 100)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
df_predict= pd.DataFrame(scalar.inverse_transform(pred_list), index= retail_lstm[-n_input:].index, columns =['Predictions'])

In [ ]:
df_test = pd.concat([retail_lstm, df_predict], axis =1)

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel("Week")
plt.ylabel("Retail Sales")
plt.title("Retail Sales - Epoch 100")
plt.plot(df_test.index, df_test['sale'])
plt.plot(df_test.index, df_test['Predictions'], color ='r')


In [ ]:
df_test.iloc[-12:,1].values

In [ ]:
df_test.iloc[-12:,0].values

### RMSE (Epoch 100)

In [ ]:
testScore = math.sqrt(mean_squared_error(df_test.iloc[-12:,0].values, df_test.iloc[-12:,1].values))
print('Test Score: %.2f RMSE' % (testScore))

## Epoch 150

In [ ]:
model = Sequential()
# Adding the input layer and LSTM layer
model.add(LSTM(200, activation= 'relu', input_shape =(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer ='adam', loss='mse')
history = model.fit_generator(generator, epochs =150)

In [ ]:
pred_list =[]
batch = train[-n_input:].reshape(1, n_input, n_features)
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:], [[pred_list[i]]], axis=1)

pred_list

## Model Loss - Epoch 150

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.title('model loss - Epoch 150')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
df_predict= pd.DataFrame(scalar.inverse_transform(pred_list), index= retail_lstm[-n_input:].index, columns =['Predictions'])

In [ ]:
df_test = pd.concat([retail_lstm, df_predict], axis =1)

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel("Week")
plt.ylabel("Retail Sales")
plt.title("Retail Sales - Epoch 150")
plt.plot(df_test.index, df_test['sale'])
plt.plot(df_test.index, df_test['Predictions'], color ='r')


### RMSE (Epoch 150)

In [ ]:
testScore = math.sqrt(mean_squared_error(df_test.iloc[-12:,0].values, df_test.iloc[-12:,1].values))
print('Test Score: %.2f RMSE' % (testScore))

## Epoch 200

In [ ]:
model = Sequential()
# Adding the input layer and LSTM layer
model.add(LSTM(200, activation= 'relu', input_shape =(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer ='adam', loss='mse')
history=model.fit_generator(generator, epochs =200)

In [ ]:
pred_list =[]
batch = train[-n_input:].reshape(1, n_input, n_features)
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:], [[pred_list[i]]], axis=1)

pred_list


## Model Loss - Epoch 200

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.title('model loss - Epoch 200')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
df_predict= pd.DataFrame(scalar.inverse_transform(pred_list), index= retail_lstm[-n_input:].index, columns =['Predictions'])

In [ ]:
df_test = pd.concat([retail_lstm, df_predict], axis =1)

In [ ]:
df_test.count()

In [ ]:
df_predict.count()

In [ ]:
df_test.info()

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel("Week")
plt.ylabel("Retail Sales")
plt.title("Retail Sales - Epoch 200")
plt.plot(df_test.index, df_test['sale'])
plt.plot(df_test.index, df_test['Predictions'], color ='r')


### RMSE (Epoch 200)

In [ ]:
testScore = math.sqrt(mean_squared_error(df_test.iloc[-12:,0].values, df_test.iloc[-12:,1].values))
print('Test Score: %.2f RMSE' % (testScore))

## Epoch 250

In [ ]:
model = Sequential()
# Adding the input layer and LSTM layer
model.add(LSTM(200, activation= 'relu', input_shape =(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer ='adam', loss='mse')
history = model.fit_generator(generator, epochs =250)

In [ ]:
pred_list =[]
batch = train[-n_input:].reshape(1, n_input, n_features)
for i in range(n_input):
    pred_list.append(model.predict(batch)[0])
    batch = np.append(batch[:, 1:,:], [[pred_list[i]]], axis=1)

pred_list

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history.history['loss'])
plt.title('model loss - Epoch 250')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
df_predict= pd.DataFrame(scalar.inverse_transform(pred_list), index= retail_lstm[-n_input:].index, columns =['Predictions'])

In [ ]:
df_test = pd.concat([retail_lstm, df_predict], axis =1)

In [ ]:
plt.figure(figsize=(20,5))
plt.xlabel("Week")
plt.ylabel("Retail Sales")
plt.title("Retail Sales - Epoch 250")
plt.plot(df_test.index, df_test['sale'])
plt.plot(df_test.index, df_test['Predictions'], color ='r')


### RMSE (Epoch 250)

In [ ]:
testScore = math.sqrt(mean_squared_error(df_test.iloc[-12:,0].values, df_test.iloc[-12:,1].values))
print('Test Score: %.2f RMSE' % (testScore))

<blank>

<blank>